# third_party - file

## Notebook运行提示
- 代码已拆分为多个小单元, 按顺序运行即可在每一步观察输出与中间变量。
- 涉及 `Path(__file__)` 或相对路径的脚本会自动注入 `__file__` 解析逻辑, Notebook 环境下也能引用原项目资源。
- 可在每个单元下追加说明或参数试验记录, 以跟踪核心算法和数据处理步骤。


In [ ]:
#from lib.include import *

In [ ]:
import os
import pickle
import sys
import pandas as pd
import shutil
import copy
from datetime import datetime

import builtins
import re

In [ ]:

class Struct(object):
	def __init__(self, is_copy=False, **kwargs):
		self.add(is_copy, **kwargs)

	def add(self, is_copy=False, **kwargs):
		#self.__dict__.update(kwargs)

		if is_copy == False:
			for key, value in kwargs.items():
				setattr(self, key, value)
		else:
			for key, value in kwargs.items():
				try:
					setattr(self, key, copy.deepcopy(value))
					#setattr(self, key, value.copy())
				except Exception:
					setattr(self, key, value)

	def drop(self,  missing=None, **kwargs):
		drop_value = []
		for key, value in kwargs.items():
			try:
				delattr(self, key)
				drop_value.append(value)
			except:
				drop_value.append(missing)
		return drop_value

	def __str__(self):
		text =''
		for k,v in self.__dict__.items():
			text += '\t%s : %s\n'%(k, str(v))
		return text

In [ ]:



# log ------------------------------------
def remove_comments(lines, token='#'):
	""" Generator. Strips comments and whitespace from input lines.
	"""

	l = []
	for line in lines:
		s = line.split(token, 1)[0].strip()
		if s != '':
			l.append(s)
	return l

In [ ]:


# def open(file, mode=None, encoding=None):
#     if mode == None: mode = 'r'
#
#     if '/' in file:
#         if 'w' or 'a' in mode:
#             dir = os.path.dirname(file)
#             if not os.path.isdir(dir):  os.makedirs(dir)
#
#     f = builtins.open(file, mode=mode, encoding=encoding)
#     return f


def remove(file):
	if os.path.exists(file): os.remove(file)

In [ ]:


def empty(dir):
	if os.path.isdir(dir):
		shutil.rmtree(dir, ignore_errors=True)
	else:
		os.makedirs(dir)

In [ ]:


# http://stackoverflow.com/questions/34950201/pycharm-print-end-r-statement-not-working
class Logger(object):
	def timestamp(self):
		h = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
		return h

	def __init__(self):
		self.terminal = sys.stdout  #stdout
		self.file = None

	def open(self, file, mode=None):
		if mode is None: mode ='w'
		self.file = open(file, mode)

	def write(self, message, is_terminal=1, is_file=1, end='\n' ):
		if '\r' in message: is_file=0

		if is_terminal == 1:
			self.terminal.write(message+end)
			self.terminal.flush()
			#time.sleep(1)

		if is_file == 1:
			self.file.write(message+end)
			self.file.flush()

	def flush(self):
		# this flush method is needed for python 3 compatibility.
		# this handles the flush command by doing nothing.
		# you might want to specify some extra behavior here.
		pass

In [ ]:

# io ------------------------------------
def write_list_to_file(list_file, strings):
	with open(list_file, 'w') as f:
		for s in strings:
			f.write('%s\n'%str(s))
	pass

In [ ]:


def read_list_from_file(list_file, comment='#'):
	with open(list_file) as f:
		lines  = f.readlines()
	strings=[]
	for line in lines:
		if comment is not None:
			s = line.split(comment, 1)[0].strip()
		else:
			s = line.strip()
		if s != '':
			strings.append(s)
	return strings

In [ ]:



def read_pickle_from_file(pickle_file):
	with open(pickle_file,'rb') as f:
		x = pickle.load(f)
	return x

In [ ]:

def write_pickle_to_file(pickle_file, x):
	with open(pickle_file, 'wb') as f:
		pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

In [ ]:



# backup ------------------------------------

#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
	assert(os.path.isdir(project_dir))
	assert(os.path.isdir(os.path.dirname(zip_file)))
	shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
	pass

In [ ]:


# etc ------------------------------------
def short_e_format(s):
	s = s.replace('e-0', 'e-').replace('e+0','e+')
	return s

In [ ]:

def time_to_str(t, mode='min'):
	if mode=='min':
		t  = int(t)/60
		hr = t//60
		min = t%60
		return '%2d hr %02d min'%(hr,min)

	elif mode=='sec':
		t   = int(t)
		min = t//60
		sec = t%60
		return '%2d min %02d sec'%(min,sec)

	else:
		raise NotImplementedError

In [ ]:


def np_float32_to_uint8(x, scale=255):
	return (x*scale).astype(np.uint8)

In [ ]:

def np_uint8_to_float32(x, scale=255):
	return (x/scale).astype(np.float32)

In [ ]:


def int_tuple(x):
	return tuple( [int(round(xx)) for xx in x] )

In [ ]:




def df_loc_by_list(df, key, values):
	df = df.loc[df[key].isin(values)]
	df = df.assign(sort = pd.Categorical(df[key], categories=values, ordered=True))
	df = df.sort_values('sort')
	#df = df.reset_index()
	df = df.drop('sort', axis=1)
	return  df

In [ ]:



import psutil

In [ ]:
def get_used_mem():
    memory = psutil.virtual_memory()
    total = memory.total / 1024.0 / 1024.0 / 1024.0
    available = memory.available / 1024.0 / 1024.0 / 1024.0 #in gb
    used = int(total-available)
    return used